In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(train, val, test):

    X_train = train.drop(['fr_yn'], 1)
    y_train = train['fr_yn']
    X_val = val.drop(['fr_yn'], 1)
    y_val = val['fr_yn']
    X_test = test.drop(['fr_yn'], 1)

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(train)]
    X_val = df_all[len(train):-len(test)]
    X_test = df_all[-len(test):]

    # X_train = X_train.fillna(-1)
    # X_val = X_val.fillna(-1)
    # X_test = X_test.fillna(-1)

    X_train = X_train.replace({-1:np.NaN})
    X_val = X_val.replace({-1:np.NaN})
    X_test = X_test.replace({-1:np.NaN})

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
def population(x):
    if x < 1000:
        return "1k"
    elif x < 10000:
        return "10k"
    elif x < 100000:
        return "100k"
    elif x < 1000000:
        return "1000k"

df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
# df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
# df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
# df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

#for imputation
s = df_train_2.isnull().sum()
c_list = np.where(s >= max(s)*0.5)
df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)


In [0]:
gas_elec_columns

['gas_engry_us_201401',
 'ele_engry_us_201401',
 'gas_engry_us_201402',
 'ele_engry_us_201402',
 'gas_engry_us_201403',
 'ele_engry_us_201403',
 'gas_engry_us_201404',
 'ele_engry_us_201404',
 'gas_engry_us_201405',
 'ele_engry_us_201405',
 'gas_engry_us_201406',
 'ele_engry_us_201406',
 'gas_engry_us_201407',
 'ele_engry_us_201407',
 'gas_engry_us_201408',
 'ele_engry_us_201408',
 'gas_engry_us_201409',
 'ele_engry_us_201409',
 'gas_engry_us_201410',
 'ele_engry_us_201410',
 'gas_engry_us_201411',
 'ele_engry_us_201411',
 'gas_engry_us_201412',
 'ele_engry_us_201412',
 'gas_engry_us_201501',
 'ele_engry_us_201501',
 'gas_engry_us_201502',
 'ele_engry_us_201502',
 'gas_engry_us_201503',
 'ele_engry_us_201503',
 'gas_engry_us_201504',
 'ele_engry_us_201504',
 'gas_engry_us_201505',
 'ele_engry_us_201505',
 'gas_engry_us_201506',
 'ele_engry_us_201506',
 'gas_engry_us_201507',
 'ele_engry_us_201507',
 'gas_engry_us_201508',
 'ele_engry_us_201508',
 'gas_engry_us_201509',
 'ele_engry_us_2

In [0]:
!pip install impyute

In [0]:
X_train

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,...,gas_engry_us_201803,ele_engry_us_201803,gas_engry_us_201804,ele_engry_us_201804,gas_engry_us_201805,ele_engry_us_201805,gas_engry_us_201806,ele_engry_us_201806,gas_engry_us_201807,ele_engry_us_201807,gas_engry_us_201808,ele_engry_us_201808,gas_engry_us_201809,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,9.0,8.0,3,69.420,69.4200,0.0,1977.0,1.0,0.0,6.0,283.85,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547,NaN,133,137.0,0.0,112,0,0,1891,29231,11322,88,0
1,NaN,NaN,3,46.290,46.2900,0.0,NaN,1.0,0.0,NaN,292.65,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388,122581.0,489,85.0,0.0,818,0,0,4533,20480,3369,7727,0
2,1.0,15.0,1,583.800,2516.7600,1446.0,2000.0,5.0,0.0,6.0,288.65,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340,618105.0,143,137.0,0.0,165,0,0,277,31197,12451,72,14
3,9.0,10.0,2,48.920,48.9200,0.0,1936.0,1.0,0.0,6.0,293.75,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179,719542.0,1585,176.0,0.0,131,0,0,438,4270,407,508,11
4,NaN,NaN,2,0.000,0.0000,0.0,NaN,NaN,NaN,NaN,285.85,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822,NaN,603,137.0,0.0,686,0,0,1702,29778,12487,707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2,423.000,423.0000,3010.0,1996.0,NaN,NaN,NaN,284.35,0.2,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435,NaN,895,139.0,0.0,1131,0,0,3591,3624,6679,914,0
59195,NaN,NaN,35,5593.711,54835.0382,0.0,NaN,171.0,0.0,NaN,287.95,1.9,20.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# start the KNN training
imputed_training=fast_knn(X_train[::10].values, k=3)

KeyboardInterrupt: ignored

In [0]:
pd.DataFrame(imputed_training)

In [0]:
from impyute.imputation.cs import mice
ㄴ
# start the MICE training
imputed_training=mice(X_train[::10].values)

In [0]:
pd.DataFrame(imputed_training)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.42,69.42,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.000000,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,7.196035e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,9.000000,10.000000,2.0,98.96,98.96,310.0,1935.000000,1.000000,0.000000,6.000000,273.25,0.6,149.586532,66.0,4.0,17.0,18.0,12.0,7.0,199.0,1.0,2583.0,2.631170e+05,66.0,137.0,0.0,46.0,1.0,2.0,135.0,31585.0,12101.0,18.0,7.0
2,9.000000,7.000000,2.0,119.02,142.86,400.0,2010.000000,3.000000,0.000000,6.000000,308.65,3.5,270.000000,19.0,4.0,13.0,18.0,12.0,6.0,199.0,1.0,2126.0,8.439902e+05,634.0,137.0,0.0,915.0,0.0,0.0,1635.0,31388.0,9312.0,696.0,0.0
3,2.000000,11.000000,9.0,979.52,979.52,0.0,2003.000000,3.000000,0.000000,1.000000,287.35,3.0,230.000000,36.0,15.0,13.0,18.0,6.0,9.0,330.0,2.0,2629.0,1.234680e+06,260.0,139.0,0.0,306.0,0.0,0.0,1745.0,9655.0,11504.0,1132.0,0.0
4,10.623864,8.331208,2.0,76.04,76.04,0.0,1982.128017,1.000000,0.000000,5.499485,298.25,1.4,230.000000,96.0,4.0,1.0,18.0,12.0,7.0,187.0,1.0,9369.0,4.997370e+05,392.0,137.0,0.0,236.0,0.0,0.0,9178.0,21101.0,6200.0,427.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5915,27.000000,11.000000,1.0,119.18,119.18,204.0,2007.000000,1.000000,0.000000,5.000000,276.75,0.8,110.000000,60.0,4.0,21.0,18.0,20.0,11.0,131.0,1.0,878.0,9.087534e+05,59.0,205.0,0.0,81.0,1.0,1.0,422.0,20812.0,37436.0,24.0,6.0
5916,28.000000,15.000000,1.0,392.01,1975.92,492.1,2005.000000,4.000000,1.000000,5.000000,277.55,1.5,180.000000,61.0,4.0,12.0,18.0,20.0,2.0,135.0,1.0,77.0,7.298834e+05,63.0,205.0,0.0,77.0,1.0,1.0,116.0,17574.0,35615.0,2.0,0.0
5917,28.000000,15.000000,1.0,158.29,603.25,277.1,2003.000000,5.000000,0.000000,5.000000,276.75,2.6,70.000000,80.0,4.0,17.0,18.0,37.0,11.0,137.0,1.0,1080.0,9.315070e+05,98.0,205.0,0.0,85.0,0.0,0.0,574.0,18427.0,36746.0,4.0,20.0
5918,9.863466,8.442613,3.0,126.32,126.32,254.0,1987.779711,3.953211,0.282866,5.657715,264.55,1.1,110.000000,60.0,4.0,17.0,18.0,12.0,9.0,323.0,1.0,1647.0,8.531521e+05,129.0,139.0,0.0,101.0,0.0,0.0,276.0,301.0,10989.0,227.0,37.0


In [0]:
!pip install datawig

     |████████████████████████████████| 81kB 4.7MB/s 
     |████████████████████████████████| 25.4MB 32.6MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import datawig


#Initialize a SimpleImputer model
for i in range(1):
    imputer = datawig.SimpleImputer(input_columns=list(X_train.columns)[1:], # column(s) containing information about the column we want to impute
                                    output_column= list(X_train.columns)[0], # the column we'd like to impute values for
                                    output_path = 'imputer_model' # stores model data and metrics
                                    )

#Fit an imputer model on the train data
imputer.fit(train_df=X_train, num_epochs=50)

#Impute missing values and return original dataframe with predictions
imputed = imputer.predict(X_val)

2019-11-18 14:51:56,768 [INFO]  
========== start: fit model
2019-11-18 14:51:56,769 [WARNING]  Already bound, ignoring bind()
2019-11-18 14:51:59,000 [INFO]  Epoch[0] Batch [0-1665]	Speed: 11992.61 samples/sec	cross-entropy=13.009201	bldng_us-accuracy=0.000000
2019-11-18 14:52:01,231 [INFO]  Epoch[0] Train-cross-entropy=8.572732
2019-11-18 14:52:01,233 [INFO]  Epoch[0] Train-bldng_us-accuracy=0.000000
2019-11-18 14:52:01,238 [INFO]  Epoch[0] Time cost=4.465
2019-11-18 14:52:01,246 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-11-18 14:52:01,373 [INFO]  Epoch[0] Validation-cross-entropy=13.124429
2019-11-18 14:52:01,374 [INFO]  Epoch[0] Validation-bldng_us-accuracy=0.000000
2019-11-18 14:52:03,589 [INFO]  Epoch[1] Batch [0-1665]	Speed: 12061.26 samples/sec	cross-entropy=7.104266	bldng_us-accuracy=0.000000
2019-11-18 14:52:05,791 [INFO]  Epoch[1] Train-cross-entropy=7.584273
2019-11-18 14:52:05,792 [INFO]  Epoch[1] Train-bldng_us-accuracy=0.000000
2019-11-18 14:52:0

In [0]:
imputed

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,bldng_us_imputed
0,9.0,8.0,2,77.13,77.13,485.00,1986.0,1.0,0.0,6.0,289.85,1.6,110.0,70.0,4.0,1.0,18.0,12.0,11.0,67,2.0,4094,1095890.0,131,178.0,0.0,735,0,0,1314,6516,10650,754,0,8.627862
1,NaN,NaN,1,105.43,98.59,315.00,NaN,1.0,0.0,NaN,293.45,0.9,110.0,96.0,4.0,1.0,18.0,12.0,6.0,64,2.0,2749,1196876.0,323,178.0,0.0,430,0,0,2197,3510,8179,804,1,12.946412
2,9.0,15.0,1,117.80,290.43,196.70,2006.0,3.0,0.0,6.0,275.25,1.2,320.0,68.0,4.0,15.0,18.0,39.0,9.0,50,1.0,784,1129360.0,127,201.0,0.0,126,0,0,158,208,17229,179,5,11.476341
3,2.0,11.0,1,336.00,336.00,1360.00,2000.0,1.0,0.0,1.0,279.35,3.4,290.0,32.0,15.0,1.0,18.0,6.0,11.0,65,1.0,4277,NaN,2,178.0,0.0,250,2,0,186,4167,7589,3279,1,-2.524520
4,9.0,7.0,3,251.02,250.45,840.00,1994.0,1.0,0.0,6.0,288.95,0.8,360.0,80.0,4.0,17.0,18.0,6.0,6.0,63,2.0,3479,904202.0,413,178.0,0.0,339,0,0,411,3307,13651,423,0,9.148700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6893,NaN,NaN,1,0.00,0.00,0.00,NaN,NaN,NaN,NaN,288.65,1.4,180.0,49.0,4.0,17.0,18.0,12.0,7.0,66,1.0,3789,1288416.0,137,201.0,0.0,214,0,0,157,283,11358,256,0,12.575102
6894,28.0,15.0,1,186.00,340.74,945.00,2007.0,2.0,0.0,5.0,287.05,1.9,110.0,85.0,4.0,13.0,18.0,20.0,6.0,56,1.0,2958,2216621.0,170,201.0,0.0,494,0,0,948,1180,21459,845,0,24.661890
6895,28.0,11.0,1,115.50,290.40,583.86,2000.0,3.0,0.0,5.0,299.45,2.3,160.0,91.0,4.0,13.0,16.0,6.0,11.0,65,1.0,2514,NaN,49,178.0,0.0,626,1,0,362,2535,2851,746,0,22.983527
6896,9.0,15.0,1,139.05,378.84,234.10,1993.0,3.0,0.0,6.0,268.55,2.9,320.0,35.0,4.0,15.0,18.0,39.0,11.0,50,1.0,1001,894800.0,115,201.0,0.0,70,0,1,115,209,16620,152,1,12.197775


In [0]:
imputed_training=fast_knn(X_train.values, k=3)
imputed_training

array([[9.00000000e+00, 8.00000000e+00, 3.00000000e+00, ...,
        1.13220000e+04, 8.80000000e+01, 0.00000000e+00],
       [1.26499905e+01, 9.05876197e+00, 3.00000000e+00, ...,
        3.36900000e+03, 7.72700000e+03, 0.00000000e+00],
       [1.00000000e+00, 1.50000000e+01, 1.00000000e+00, ...,
        1.24510000e+04, 7.20000000e+01, 1.40000000e+01],
       ...,
       [5.40794606e+00, 1.03788131e+01, 2.00000000e+00, ...,
        4.87800000e+03, 7.90000000e+02, 0.00000000e+00],
       [2.70000000e+01, 1.00000000e+00, 2.00000000e+00, ...,
        3.57200000e+03, 6.13000000e+02, 0.00000000e+00],
       [3.00000000e+01, 1.00000000e+00, 3.00000000e+00, ...,
        1.17690000e+04, 4.15000000e+02, 0.00000000e+00]])

In [0]:
pd.DataFrame(imputed_training)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,12.649990,9.058762,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.000000,5.215133,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,5.407946,10.378813,2.0,2436.820,2468.8000,6512.0,2002.829298,4.000000,0.000000,2.348822,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [0]:
imputed_training=mice(X_train.values)
pd.DataFrame(imputed_training)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.992864e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,11.471413,7.158777,3.0,46.290,46.2900,0.0,1978.864179,1.000000,0.000000,5.372451,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,17.924120,8.970214,2.0,0.000,0.0000,0.0,2000.875212,3.739419,0.162773,4.032174,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,7.110279e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,-1.248431,-0.209189,5.063719,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,7.160258e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,-7.025766,10.476746,35.0,5593.711,54835.0382,0.0,2002.020333,171.000000,0.000000,4.708207,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,12.404926,8.445795,2.0,2436.820,2468.8000,6512.0,1991.025486,4.000000,0.000000,3.509458,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,4.802691e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.632858e+04,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [0]:
pd.DataFrame(imputed_training).to_csv(path + 'imputed2_34f.csv',index=False, header=False)

In [5]:
X_train_2 = pd.read_csv(path + "imputed_34f.csv", header=None)
X_train_2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,12.649990,9.058762,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.000000,5.215133,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,5.407946,10.378813,2.0,2436.820,2468.8000,6512.0,2002.829298,4.000000,0.000000,2.348822,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [0]:
c_list = ['bldng_us',
 'bldng_archtctr',
 'bldng_cnt',
 'bldng_ar',
 'ttl_ar',
 'lnd_ar',
 'dt_of_athrztn',
 'ttl_grnd_flr',
 'ttl_dwn_flr',
 'bldng_us_clssfctn',
 'tmprtr',
 'wnd_spd',
 'wnd_drctn',
 'hmdt',
 'jmk',
 'rgnl_ar_nm',
 'rgnl_ar_nm2',
 'lnd_us_sttn_nm',
 'rd_sd_nm',
 'emd_nm',
 'hm_cnt',
 'fr_sttn_dstnc',
 'bldng_ar_prc',
 'fr_wthr_fclt_dstnc',
 'fr_mn_cnt',
 'mlt_us_yn',
 'cctv_dstnc',
 'fr_wthr_fclt_in_100m',
 'cctv_in_100m',
 'tbc_rtl_str_dstnc',
 'sft_emrgnc_bll_dstnc',
 'ahsm_dstnc',
 'no_tbc_zn_dstnc',
 'bldng_cnt_in_50m']

In [7]:
X_train[c_list]

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,9.0,8.0,3,69.420,69.4200,0.0,1977.0,1.0,0.0,6.0,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547,NaN,133,137.0,0.0,112,0,0,1891,29231,11322,88,0
1,NaN,NaN,3,46.290,46.2900,0.0,NaN,1.0,0.0,NaN,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388,122581.0,489,85.0,0.0,818,0,0,4533,20480,3369,7727,0
2,1.0,15.0,1,583.800,2516.7600,1446.0,2000.0,5.0,0.0,6.0,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340,618105.0,143,137.0,0.0,165,0,0,277,31197,12451,72,14
3,9.0,10.0,2,48.920,48.9200,0.0,1936.0,1.0,0.0,6.0,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179,719542.0,1585,176.0,0.0,131,0,0,438,4270,407,508,11
4,NaN,NaN,2,0.000,0.0000,0.0,NaN,NaN,NaN,NaN,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822,NaN,603,137.0,0.0,686,0,0,1702,29778,12487,707,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2,423.000,423.0000,3010.0,1996.0,NaN,NaN,NaN,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435,NaN,895,139.0,0.0,1131,0,0,3591,3624,6679,914,0
59195,NaN,NaN,35,5593.711,54835.0382,0.0,NaN,171.0,0.0,NaN,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137,1849455.0,88,139.0,0.0,108,0,0,2629,77,1285,131,0
59196,NaN,NaN,2,2436.820,2468.8000,6512.0,NaN,4.0,0.0,NaN,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875,NaN,25,139.0,0.0,968,2,0,4271,6139,4878,790,0
59197,27.0,1.0,2,283.200,207.7100,0.0,2005.0,2.0,0.0,5.0,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579,NaN,510,139.0,0.0,177,0,0,1387,10142,3572,613,0


In [8]:
X_train_2.columns = c_list
X_train_2

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,12.649990,9.058762,35.0,5593.711,54835.0382,0.0,1984.654245,171.000000,0.000000,5.215133,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0
59196,5.407946,10.378813,2.0,2436.820,2468.8000,6512.0,2002.829298,4.000000,0.000000,2.348822,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0
59197,27.000000,1.000000,2.0,283.200,207.7100,0.0,2005.000000,2.000000,0.000000,5.000000,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0


In [9]:
X_train[c_list] = X_train_2
X_train

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,...,gas_engry_us_201803,ele_engry_us_201803,gas_engry_us_201804,ele_engry_us_201804,gas_engry_us_201805,ele_engry_us_201805,gas_engry_us_201806,ele_engry_us_201806,gas_engry_us_201807,ele_engry_us_201807,gas_engry_us_201808,ele_engry_us_201808,gas_engry_us_201809,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,9.000000,8.000000,3.0,69.420,69.4200,0.0,1977.000000,1.000000,0.000000,6.000000,283.85,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,12.649990,9.058762,3.0,46.290,46.2900,0.0,1984.654245,1.000000,0.000000,5.215133,292.65,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.000000,15.000000,1.0,583.800,2516.7600,1446.0,2000.000000,5.000000,0.000000,6.000000,288.65,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.000000,10.000000,2.0,48.920,48.9200,0.0,1936.000000,1.000000,0.000000,6.000000,293.75,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,14.538810,6.531796,2.0,0.000,0.0000,0.0,1968.814380,1.000000,0.000000,5.692288,285.85,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.000000,1.000000,2.0,423.000,423.0000,3010.0,1996.000000,3.121446,0.083291,5.215133,284.35,0.2,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1

In [0]:
df = df_train._get_numeric_data()
a = df.columns

In [0]:
b = df_train.columns

In [12]:
temp_list = []
for x in b:
    if x not in a:
        temp_list.append(x)
temp_list

['dt_of_fr',
 'bldng_us',
 'bldng_archtctr',
 'dt_of_athrztn',
 'bldng_us_clssfctn',
 'jmk',
 'rgnl_ar_nm',
 'rgnl_ar_nm2',
 'lnd_us_sttn_nm',
 'rd_sd_nm',
 'emd_nm',
 'mlt_us_yn',
 'trgt_crtr',
 'fr_fghtng_fclt_spcl_css_5_yn',
 'fr_fghtng_fclt_spcl_css_6_yn',
 'us_yn',
 'dngrs_thng_yn',
 'slf_fr_brgd_yn',
 'blk_dngrs_thng_mnfctr_yn',
 'cltrl_hrtg_yn']

In [13]:
X_train[temp_list]

KeyError: ignored

In [14]:
temp_list_2 = []
for x in temp_list:
    if x not in ['us_yn', 'dngrs_thng_yn', 'fr_yn', 'dt_of_fr', 'trgt_crtr', 'fr_fghtng_fclt_spcl_css_6_yn', 'fr_fghtng_fclt_spcl_css_5_yn', 'blk_dngrs_thng_mnfctr_yn', 'cltrl_hrtg_yn', 'slf_fr_brgd_yn']:
        temp_list_2.append(x)
temp_list_2

['bldng_us',
 'bldng_archtctr',
 'dt_of_athrztn',
 'bldng_us_clssfctn',
 'jmk',
 'rgnl_ar_nm',
 'rgnl_ar_nm2',
 'lnd_us_sttn_nm',
 'rd_sd_nm',
 'emd_nm',
 'mlt_us_yn']

In [0]:
X_train[temp_list_2] = np.round(X_train[temp_list_2])

In [16]:
X_train

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,gas_engry_us_201401,ele_engry_us_201401,gas_engry_us_201402,ele_engry_us_201402,gas_engry_us_201403,ele_engry_us_201403,gas_engry_us_201404,ele_engry_us_201404,gas_engry_us_201405,ele_engry_us_201405,gas_engry_us_201406,ele_engry_us_201406,gas_engry_us_201407,ele_engry_us_201407,gas_engry_us_201408,ele_engry_us_201408,gas_engry_us_201409,ele_engry_us_201409,gas_engry_us_201410,ele_engry_us_201410,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,...,gas_engry_us_201803,ele_engry_us_201803,gas_engry_us_201804,ele_engry_us_201804,gas_engry_us_201805,ele_engry_us_201805,gas_engry_us_201806,ele_engry_us_201806,gas_engry_us_201807,ele_engry_us_201807,gas_engry_us_201808,ele_engry_us_201808,gas_engry_us_201809,ele_engry_us_201809,gas_engry_us_201810,ele_engry_us_201810,gas_engry_us_201811,ele_engry_us_201811,gas_engry_us_201812,ele_engry_us_201812,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
0,9.0,8.0,3.0,69.420,69.4200,0.0,1977.0,1.000000,0.000000,6.0,283.85,0.9,200.0,96.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0
1,13.0,9.0,3.0,46.290,46.2900,0.0,1985.0,1.000000,0.000000,5.0,292.65,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0
2,1.0,15.0,1.0,583.800,2516.7600,1446.0,2000.0,5.000000,0.000000,6.0,288.65,2.0,90.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0
3,9.0,10.0,2.0,48.920,48.9200,0.0,1936.0,1.000000,0.000000,6.0,293.75,0.4,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0
4,15.0,7.0,2.0,0.000,0.0000,0.0,1969.0,1.000000,0.000000,6.0,285.85,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2.0,423.000,423.0000,3010.0,1996.0,3.121446,0.083291,5.0,284.35,0.2,0.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0
59195,13.0,

In [0]:
X_train.to_csv(path + "imputed_f154.csv")

In [0]:
X_train = pd.read_csv(path + "imputed_f154.csv")

In [0]:
X_train_t1 = X_train[c_list+gas_elec_columns[:20]]

In [19]:
X_train_t1.isnull().sum()

bldng_us                    0
bldng_archtctr              0
bldng_cnt                   0
bldng_ar                    0
ttl_ar                      0
lnd_ar                      0
dt_of_athrztn               0
ttl_grnd_flr                0
ttl_dwn_flr                 0
bldng_us_clssfctn           0
tmprtr                      0
wnd_spd                     0
wnd_drctn                   0
hmdt                        0
jmk                         0
rgnl_ar_nm                  0
rgnl_ar_nm2                 0
lnd_us_sttn_nm              0
rd_sd_nm                    0
emd_nm                      0
hm_cnt                      0
fr_sttn_dstnc               0
bldng_ar_prc                0
fr_wthr_fclt_dstnc          0
fr_mn_cnt                   0
mlt_us_yn                   0
cctv_dstnc                  0
fr_wthr_fclt_in_100m        0
cctv_in_100m                0
tbc_rtl_str_dstnc           0
sft_emrgnc_bll_dstnc        0
ahsm_dstnc                  0
no_tbc_zn_dstnc             0
bldng_cnt_

In [21]:
!pip install impyute

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# start the KNN training
imputed_training=fast_knn(X_train_t1.values, k=3)

In [23]:
pd.DataFrame(imputed_training)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,9.0,8.0,3.0,69.420,69.4200,0.0,1977.0,1.000000,0.000000,6.0,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000
1,13.0,9.0,3.0,46.290,46.2900,0.0,1985.0,1.000000,0.000000,5.0,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0,12667.549276,2302.02527,13262.269435,2231.645297,10505.810741,1931.644158,8313.110696,1768.842052,4317.892688,1640.096464,3160.456702,1647.034253,1256.941299,1738.445474,3608.5754,1931.299766,2015.16168,1860.793187,2254.52565,2110.401891
2,1.0,15.0,1.0,583.800,2516.7600,1446.0,2000.0,5.000000,0.000000,6.0,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000
3,9.0,10.0,2.0,48.920,48.9200,0.0,1936.0,1.000000,0.000000,6.0,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000
4,15.0,7.0,2.0,0.000,0.0000,0.0,1969.0,1.000000,0.000000,6.0,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0,12667.549276,2302.02527,13262.269435,2231.645297,10505.810741,1931.644158,8313.110696,1768.842052,4317.892688,1640.096464,3160.456702,1647.034253,1256.941299,1738.445474,3608.5754,1931.299766,2015.16168,1860.793187,2254.52565,2110.401891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2.0,423.000,423.0000,3010.0,1996.0,3.121446,0.083291,5.0,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000
59195,13.0,9.0,35.0,5593.711,54835.0382,0.0,1985.0,171.000000,0.000000,5.0,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0,12667.549276,2302.02527,13262.269435,2231.645297,10505.810741,1931.644158,8313.110696,1768.842052,4317.892688,1640.096464,3160.456702,1647.034253,1256.941299,1738.445474,3608.5754,1931.299766,2015.16168,1860.793187,2254.52565,2110.401891
59196,5.0,10.0,2.0,2436.820,2468.8000,6512.0,2003.0,4.000000,0.000000,2.0,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0,12667.549276,2302.02527,13262.269435,2231.645297,10505.810741,1931.644158,8313.110696,1768.842052,4317.892688,1640.096464,3160.456702,1647.034253,1256.941299,1738.445474,3608.5754,1931.299766,2015.16168,1860.793187,2254.52565,2110.401891
59197,27.0,1.0,2.0,283.200,207.7100,0.0,2005.0,2.000000,0.000000,5.0,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0

In [0]:
pd.DataFrame(imputed_training).to_csv(path + "imputed_34f+20_1_eg.csv", index=None, header=None)

In [28]:
X_train_t2 = X_train[c_list+gas_elec_columns[20:40]]
X_train_t2

,bldng_us,bldng_archtctr,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,bldng_us_clssfctn,tmprtr,wnd_spd,wnd_drctn,hmdt,jmk,rgnl_ar_nm,rgnl_ar_nm2,lnd_us_sttn_nm,rd_sd_nm,emd_nm,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,mlt_us_yn,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,gas_engry_us_201411,ele_engry_us_201411,gas_engry_us_201412,ele_engry_us_201412,gas_engry_us_201501,ele_engry_us_201501,gas_engry_us_201502,ele_engry_us_201502,gas_engry_us_201503,ele_engry_us_201503,gas_engry_us_201504,ele_engry_us_201504,gas_engry_us_201505,ele_engry_us_201505,gas_engry_us_201506,ele_engry_us_201506,gas_engry_us_201507,ele_engry_us_201507,gas_engry_us_201508,ele_engry_us_201508
0,9.0,8.0,3.0,69.420,69.4200,0.0,1977.0,1.000000,0.000000,6.0,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13.0,9.0,3.0,46.290,46.2900,0.0,1985.0,1.000000,0.000000,5.0,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,15.0,1.0,583.800,2516.7600,1446.0,2000.0,5.000000,0.000000,6.0,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9.0,10.0,2.0,48.920,48.9200,0.0,1936.0,1.000000,0.000000,6.0,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,15.0,7.0,2.0,0.000,0.0000,0.0,1969.0,1.000000,0.000000,6.0,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2.0,423.000,423.0000,3010.0,1996.0,3.121446,0.083291,5.0,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59195,13.0,9.0,35.0,5593.711,54835.0382,0.0,1985.0,171.000000,0.000000,5.0,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59196,5.0,10.0,2.0,2436.820,2468.8000,6512.0,2003.0,4.000000,0.000000,2.0,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59197,27.0,1.0,2.0,283.200,207.7100,0.0,2005.0,2.000000,0.000000,5.0,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS

# start the KNN training
imputed_training=fast_knn(X_train_t2.values, k=3)

In [34]:
pd.DataFrame(imputed_training)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,9.0,8.0,3.0,69.420,69.4200,0.0,1977.0,1.000000,0.000000,6.0,283.85,0.9,200.0,96.0,4.0,13.0,18.0,12.0,7.0,199.0,1.0,4547.0,6.397596e+05,133.0,137.0,0.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
1,13.0,9.0,3.0,46.290,46.2900,0.0,1985.0,1.000000,0.000000,5.0,292.65,0.5,20.0,74.0,4.0,1.0,18.0,12.0,6.0,141.0,2.0,6388.0,1.225810e+05,489.0,85.0,0.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0,5479.170789,2636.116041,9412.83288,4570.554969,5544.479316,4273.816782,13639.704725,6528.678822,36300.352046,10765.41081,25294.777658,10211.102916,14275.4518,9988.028022,8831.543551,13968.88127,5552.546826,15708.724176,5215.213707,15997.452875
2,1.0,15.0,1.0,583.800,2516.7600,1446.0,2000.0,5.000000,0.000000,6.0,288.65,2.0,90.0,21.0,3.0,17.0,18.0,25.0,11.0,199.0,1.0,3340.0,6.181050e+05,143.0,137.0,0.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
3,9.0,10.0,2.0,48.920,48.9200,0.0,1936.0,1.000000,0.000000,6.0,293.75,0.4,0.0,91.0,4.0,15.0,18.0,12.0,7.0,261.0,2.0,179.0,7.195420e+05,1585.0,176.0,0.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
4,15.0,7.0,2.0,0.000,0.0000,0.0,1969.0,1.000000,0.000000,6.0,285.85,0.6,360.0,89.0,13.0,14.0,18.0,31.0,7.0,199.0,1.0,4822.0,6.397596e+05,603.0,137.0,0.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0,5479.170789,2636.116041,9412.83288,4570.554969,5544.479316,4273.816782,13639.704725,6528.678822,36300.352046,10765.41081,25294.777658,10211.102916,14275.4518,9988.028022,8831.543551,13968.88127,5552.546826,15708.724176,5215.213707,15997.452875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,10.0,1.0,2.0,423.000,423.0000,3010.0,1996.0,3.121446,0.083291,5.0,284.35,0.2,0.0,41.0,6.0,1.0,18.0,35.0,6.0,331.0,1.0,3435.0,6.397596e+05,895.0,139.0,0.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
59195,13.0,9.0,35.0,5593.711,54835.0382,0.0,1985.0,171.000000,0.000000,5.0,287.95,1.9,20.0,72.0,4.0,1.0,18.0,22.0,9.0,331.0,1.0,1137.0,1.849455e+06,88.0,139.0,0.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0,5479.170789,2636.116041,9412.83288,4570.554969,5544.479316,4273.816782,13639.704725,6528.678822,36300.352046,10765.41081,25294.777658,10211.102916,14275.4518,9988.028022,8831.543551,13968.88127,5552.546826,15708.724176,5215.213707,15997.452875
59196,5.0,10.0,2.0,2436.820,2468.8000,6512.0,2003.0,4.000000,0.000000,2.0,286.45,3.2,230.0,58.0,15.0,1.0,18.0,6.0,11.0,326.0,2.0,5875.0,6.397596e+05,25.0,139.0,0.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0,5479.170789,2636.116041,9412.83288,4570.554969,5544.479316,4273.816782,13639.704725,6528.678822,36300.352046,10765.41081,25294.777658,10211.102916,14275.4518,9988.028022,8831.543551,13968.88127,5552.546826,15708.724176,5215.213707,15997.452875
59197,27.0,1.0,2.0,283.200,207.7100,0.0,2005.0,2.000000,0.000000,5.0,291.35,1.4,360.0,95.0,3.0,4.0,18.0,14.0,3.0,325.0,2.0,8579.0,6.397596e+05,510.0,139.0,0.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.00

In [0]:
pd.DataFrame(imputed_training).to_csv(path + "imputed_34f+20_2_eg.csv", index=None, header=None)

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS


X_train_t3 = X_train[c_list+gas_elec_columns[40:60]]
X_train_t3

# start the KNN training
imputed_training=fast_knn(X_train_t3.values, k=3)
pd.DataFrame(imputed_training).to_csv(path + "imputed_34f+20_3_eg.csv", index=None, header=None)

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS


X_train_t4 = X_train[c_list+gas_elec_columns[60:80]]
X_train_t4

# start the KNN training
imputed_training=fast_knn(X_train_t4.values, k=3)
pd.DataFrame(imputed_training).to_csv(path + "imputed_34f+20_4_eg.csv", index=None, header=None)

In [0]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000) #Increase the recursion limit of the OS


X_train_t5 = X_train[c_list+gas_elec_columns[80:100]]
X_train_t5

# start the KNN training
imputed_training=fast_knn(X_train_t5.values, k=3)
pd.DataFrame(imputed_training).to_csv(path + "imputed_34f+20_5_eg.csv", index=None, header=None)